# Step 2: Private Prediction using Syft Keras (Server)

In [1]:
#import torch
import tensorflow as tf
import numpy as np
import syft
import tf_encrypted as tfe
# hook = syft.TorchHook(torch)
# alice = syft.VirtualWorker(hook, id="alice")  # alice computes on behalf of data owner
# bob = syft.VirtualWorker(hook, id="bob")      # bob computes on behalf of model owner
# carol = syft.VirtualWorker(hook, id="carol")  # carol acts as crypto producer

In [3]:
#import tf_encrypted as tfe
from syft.keras.model import Sequential
from syft.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

In [4]:
task_classes = 10
#task_shape = [None, 28, 28, 1]
task_shape = [1,128]
pre_trained_weights = 'short-conv-mnist.h5'

In [5]:
model = Sequential()

# model.add(Conv2D(10, (3, 3), input_shape=task_shape[1:]))
# model.add(AveragePooling2D((2, 2)))
# model.add(Activation('relu'))
# model.add(Conv2D(32, (3, 3)))
# model.add(AveragePooling2D((2, 2)))
# model.add(Activation('relu'))
# model.add(Conv2D(64, (3, 3)))
# model.add(AveragePooling2D((2, 2)))
# model.add(Activation('relu'))
# model.add(Flatten())
# model.add(Dense(task_classes, name="logit"))

#model.add(Dense(32, input_shape=task_shape[1:]))
model.add(Dense(32, batch_size=1, input_shape=task_shape[1:]))
model.add(Dense(10, name='logit'))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# Load pre-trained weights
#model.load_weights(pre_trained_weights)

In [7]:
#data = np.ones([1, 28, 28, 1])
data = np.ones([1, 128])

In [8]:
public_prediction = model.predict(data)
print("Public Prediction:", public_prediction)

Public Prediction: [[ 2.4663649  -2.2198696   1.211142    2.5997243  -0.4608728  -2.2953916
   2.1530936  -0.63531    -0.17487171  1.2820201 ]]


### Secure the model by sharing the weights

In [9]:
# prot = tfe.SecureNN(player_0=alice,
#                     player_1=bob,
#                     comparison_helper=carol,
#                     crypto_producer=carol)

securenn_prot = tfe.protocol.SecureNN()

tfe_model = model.share(prot=securenn_prot)

(1, 128)
None


In [10]:
with securenn_prot:
    x = tfe.define_private_variable(data)

    y = tfe_model(x)

In [11]:
with tfe.Session() as sess:
    sess.run(tf.global_variables_initializer())
    actual = sess.run(y.reveal())

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2']


In [12]:
print("Private Prediction: ", actual)

Private Prediction:  [[ 2.46303917 -2.218107    1.21002896  2.59792715 -0.46029569 -2.29233349
   2.15104405 -0.63389727 -0.173754    1.27937814]]
